In [2]:
import numpy as np
import tobii_research as tr
import time
import random
import os
import pylsl as lsl
import sys

In [3]:
# Find Eye Tracker and Apply License (edit to suit actual tracker serial no)
ft = tr.find_all_eyetrackers()
if len(ft) == 0:
    print ("No Eye Trackers found!?")
    exit(1)
# Pick first tracker
mt = ft[0]
print ("Found Tobii Tracker at '%s'" % (mt.address)) 

Found Tobii Tracker at 'tet-tcp://169.254.10.243'


In [ ]:
channels = 3 # count of the below channels, incl. those that are 3 or 2 long
external_stuff = [
    ('device_time_stamp', 1),
    ('value',  1),
    ('system_time_stamp', 1),
]
    

def unpack_external_signal(external_signal_data):
    x = []
    for s in external_stuff:
        d = external_signal_data[s[0]]
        if isinstance(d, tuple):
            x = x + list(d)
        else:
            x.append(d)
    return x

last_report = 0
N = 0

def external_signal_callback(external_signal_data):
    ## For each trigger send - 8 bits of data are sent separately
    print(external_signal_data['value'])
    try:
        global last_report
        global outlet
        global N
        global halted

        sts = external_signal_data['system_time_stamp'] / 1000000.

        outlet.push_sample(unpack_external_signal(external_signal_data), sts)
        
        if sts > last_report + 5:
            sys.stdout.write("%14.3f: %10d packets\r" % (sts, N))
            last_report = sts
        N += 1

        print(unpack_external_signal(external_signal_data))
    except:
        print("Error in callback: ")
        print(sys.exc_info())
        halted = True

def start_external_tracking():
    mt.subscribe_to(tr.EYETRACKER_EXTERNAL_SIGNAL, external_signal_callback, as_dictionary=True)
    return True

def end_external_tracking():
    mt.unsubscribe_from(tr.EYETRACKER_EXTERNAL_SIGNAL, external_signal_callback)
    return True

halted = False


# Set up lsl stream
def setup_lsl():
    global channels
    global external_stuff

    info = lsl.StreamInfo('Tobii_TTL', 'ET', 3, lsl.IRREGULAR_RATE, 'float32', mt.address)
    info.desc().append_child_value("manufacturer", "Tobii")
    channels = info.desc().append_child("channels")
    cnt = 0
    for s in external_stuff:
        if s[1]==1:
            cnt += 1
            channels.append_child("channel") \
                    .append_child_value("label", s[0]) \
                    .append_child_value("unit", "device") \
                    .append_child_value("type", 'ET')
        else:
            for i in range(s[1]):
                cnt += 1
                channels.append_child("channel") \
                        .append_child_value("label", "%s_%d" % (s[0], i)) \
                        .append_child_value("unit", "device") \
                        .append_child_value("type", 'ET')

    outlet = lsl.StreamOutlet(info)

    return outlet

outlet = setup_lsl()

#Main loop; run until escape is pressed
print ("%14.3f: LSL Running; press CTRL-C repeatedly to stop"  % lsl.local_clock())
start_external_tracking()
try:
    while not halted:
        time.sleep(1)
        keys = ()  # event.getKeys()
        if len(keys) != 0:
            if keys[0]=='escape':
                halted = True

        if halted:
            break

        # print lsl.local_clock()

except:
    print ("Halting...")

print ("terminating tracking now")
end_external_tracking()


   2283612.395: LSL Running; press CTRL-C repeatedly to stop
179
[515455304970, 179, 2283612659070]
187
[515494362479, 187, 2283651716462]
251
[515494362569, 251, 2283651716552]
179
[515509446045, 179, 2283666800009]
251
[515509446280, 251, 2283666800244]
187
[515509446438, 187, 2283666800402]
179
[515509530014, 179, 2283666883978]
251
[515510030436, 251, 2283667384401]
187
[515510030574, 187, 2283667384539]
179
[515510114168, 179, 2283667468133]
187
[515511614675, 187, 2283668968643]
179
[515511698397, 179, 2283669052365]
187
[515513198910, 187, 2283670552882]
179
[515513282632, 179, 2283670636604]
187
[515514783140, 187, 2283672137114]
179
[515514866863, 179, 2283672220837]
187
[515515368060, 187, 2283672722033]
179
[515515451779, 179, 2283672805752]
187
[515515952201, 187, 2283673306174]
179
[515516035932, 179, 2283673389905]
187
[515516536441, 187, 2283673890413]
179
[515516620165, 179, 2283673974137]
187
[515518120768, 187, 2283675474738]
179
[515518204485, 179, 2283675558455]
187

251
[516706867287, 251, 2284864220652]
187
[516706867426, 187, 2284864220791]
179
[516706951027, 179, 2284864304392]
187
[516708451508, 187, 2284865804866]
179
[516708535244, 179, 2284865888602]
187
[516709035865, 187, 2284866389220]
179
[516709119587, 179, 2284866472942]
187
[516709620060, 187, 2284866973412]
179
[516709704198, 179, 2284867057550]
187
[516710204690, 187, 2284867558039]
179
[516710288404, 179, 2284867641753]
187
[516710788902, 187, 2284868142248]
179
[516710872626, 179, 2284868225972]
187
[516711373097, 187, 2284868726441]
179
[516711457760, 179, 2284868811103]
187
[516711958911, 187, 2284869312253]
179
[516712042633, 179, 2284869395975]
187
[516713543114, 187, 2284870896456]
179
[516713626835, 179, 2284870980177]
187
[516715127318, 187, 2284872480659]
179
[516715210994, 179, 2284872564335]
187
[516715711534, 187, 2284873064875]
179
[516715795243, 179, 2284873148584]
187
[516717296159, 187, 2284874649501]
179
[516717379887, 179, 2284874733229]
187
[516718880522, 187, 2

187
[519184648934, 187, 2287342001393]
179
[519184714703, 179, 2287342067162]
187
[519185232268, 187, 2287342584727]
179
[519185298900, 179, 2287342651359]
187
[519186415606, 187, 2287343768065]
179
[519186598940, 179, 2287343951399]
187
[519186967791, 187, 2287344320250]
179
[519187051501, 179, 2287344403960]
187
[519187598943, 187, 2287344951403]
179
[519187782276, 179, 2287345134736]
187
[519188182277, 187, 2287345534737]
179
[519188365612, 179, 2287345718072]
187
[519188765613, 187, 2287346118073]
179
[519188948946, 179, 2287346301406]
187
[519189304457, 187, 2287346656917]
179
[519189388194, 179, 2287346740654]
187
[519189888650, 187, 2287347241108]
179
[519190082283, 179, 2287347434740]
187
[519190472889, 187, 2287347825345]
179
[519190682284, 179, 2287348034739]
187
[519191132285, 187, 2287348484738]
179
[519191140805, 179, 2287348493258]
187
[519191648954, 187, 2287349001404]
179
[519191832287, 179, 2287349184736]
187
[519192282288, 187, 2287349634735]
179
[519192309771, 179, 2

187
[519892865739, 187, 2288050217975]
179
[519892949540, 179, 2288050301776]
187
[519893950030, 187, 2288051302265]
179
[519894033744, 179, 2288051385979]
187
[519895034219, 187, 2288052386453]
179
[519895117948, 179, 2288052470182]
187
[519896118428, 187, 2288053470661]
179
[519896202178, 179, 2288053554411]
187
[519897202642, 187, 2288054554875]
179
[519897286269, 179, 2288054638502]
187
[519898286612, 187, 2288055638845]
179
[519898370317, 179, 2288055722550]
187
[519899370773, 187, 2288056723006]
179
[519899454487, 179, 2288056806720]
187
[519900454913, 187, 2288057807145]
179
[519900538630, 179, 2288057890862]
187
[519901539102, 187, 2288058891334]
179
[519901622828, 179, 2288058975060]
187
[519902623307, 187, 2288059975539]
179
[519902707028, 179, 2288060059260]
187
[519903707545, 187, 2288061059776]
179
[519903792174, 179, 2288061144405]
187
[519904792604, 187, 2288062144833]
179
[519904876322, 179, 2288062228551]
187
[519905876798, 187, 2288063229024]
179
[519905960563, 179, 2